# Feature Engineering and a few improvments

In this notebook, I'll do some feature engineering. In the next notebook, I'll use a technique called Automated Feature Engineering. 

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
os.getcwd()

'C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\notebooks'

In [3]:
# Importing utils 
os.chdir('C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\code')

from utils import *

# Data directory
os.chdir('C:\\Users\\hugo_\\OneDrive\\Documentos\\DataScience\\Repos\\kaggle_credit_risk\\data\\treated')

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,...,17255.559844,18101.079844,18101.079844,0.255556,0.239583,0.0,0.000000,18.719101,0.010417,0.010417
3,3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,7680.352041,7968.609184,7968.609184,0.045455,2.387755,0.0,2.613636,19.547619,0.000000,0.000000
4,4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)

I'm gonna drop columns that have more than 70% missing values.

In [8]:
# Dropping missing columns

train, DROP_COLUMNS = drop_missings_columns(train, treshold = 0.7)
test.drop(DROP_COLUMNS, axis=1, inplace=True)

## Pipeline to use tree based models

Now I'm gonna separate the dataframes between numerical and categoricals. Then, I'll fill those missing data adopting some strategy (let's start with mean and see where it takes us), and for categoricals, mode.

In [9]:
train_num = train.select_dtypes(exclude='object')
train_cat = train.select_dtypes(include='object')

In [10]:
test_num = test.select_dtypes(exclude='object')
test_cat = test.select_dtypes(include='object')

In [11]:
train_num = fill_numerical_missings(train_num, 'mean')
test_num = fill_numerical_missings(test_num, 'mean')

train_cat = fill_categorical_missings(train_cat)
test_cat = fill_categorical_missings(test_cat)

For categorical data, let's create some catcodes.

In [12]:
train_cat, mappers = get_catcodes(train_cat)

In [13]:
mappers

{'NAME_CONTRACT_TYPE': {'Cash loans': 0, 'Revolving loans': 1},
 'CODE_GENDER': {'M': 0, 'F': 1, 'XNA': 2},
 'FLAG_OWN_CAR': {'N': 0, 'Y': 1},
 'FLAG_OWN_REALTY': {'Y': 0, 'N': 1},
 'NAME_TYPE_SUITE': {'Unaccompanied': 0,
  'Family': 1,
  'Spouse, partner': 2,
  'Children': 3,
  'Other_A': 4,
  'Other_B': 5,
  'Group of people': 6},
 'NAME_INCOME_TYPE': {'Working': 0,
  'State servant': 1,
  'Commercial associate': 2,
  'Pensioner': 3,
  'Unemployed': 4,
  'Student': 5,
  'Businessman': 6,
  'Maternity leave': 7},
 'NAME_EDUCATION_TYPE': {'Secondary / secondary special': 0,
  'Higher education': 1,
  'Incomplete higher': 2,
  'Lower secondary': 3,
  'Academic degree': 4},
 'NAME_FAMILY_STATUS': {'Single / not married': 0,
  'Married': 1,
  'Civil marriage': 2,
  'Widow': 3,
  'Separated': 4,
  'Unknown': 5},
 'NAME_HOUSING_TYPE': {'House / apartment': 0,
  'Rented apartment': 1,
  'With parents': 2,
  'Municipal apartment': 3,
  'Office apartment': 4,
  'Co-op apartment': 5},
 'OCCUPAT

In [14]:
for col, values in mappers.items():
    test_cat[col] = test_cat[col].map(values)

Great! So now I'm gonna do some feature engineering.

# Feature Engineering

I'll create some features using polinomial features from sklearn. Them I'm gonna try to use some domain knowledge in order to develop more powerful features.

I'll expand the most correlated features using this strategy. I hope that this brings some gains into the solution.

In [15]:
from sklearn.preprocessing import PolynomialFeatures

# Selecting some features to perform polinomial expansion
poly_feats_train = train_num[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
poly_feats_test = test_num[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# Creating transformer
poly_transformer = PolynomialFeatures(degree = 3)

# Transforming features
poly_transformer.fit(poly_feats_train)

poly_feats_train = poly_transformer.transform(poly_feats_train)
poly_feats_test = poly_transformer.transform(poly_feats_test)

print('Shape of transformed features: '+ str(poly_feats_train.shape))

Shape of transformed features: (307511, 35)


Good! Now let's turn it back to pandas dataframe and concatenate them with our original training/testing data.

In [16]:
poly_feats_train = pd.DataFrame(poly_feats_train, 
                                columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

poly_feats_test = pd.DataFrame(poly_feats_test, 
                                columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

In [17]:
poly_feats_train.head()

,1,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,EXT_SOURCE_1^2,EXT_SOURCE_1 EXT_SOURCE_2,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_1 DAYS_BIRTH,EXT_SOURCE_2^2,...,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,1.0,0.083037,0.262949,0.139376,-9461.0,0.006895,0.021834,0.011573,-785.612748,0.069142,...,0.018181,0.009637,-654.152107,0.005108,-346.733022,2.353667e+07,0.002707,-183.785678,1.247560e+07,-8.468590e+11
1,1.0,0.311267,0.622246,0.510853,-16765.0,0.096887,0.193685,0.159012,-5218.396475,0.387190,...,0.240927,0.197797,-6491.237078,0.162388,-5329.192190,1.748916e+08,0.133318,-4375.173647,1.435830e+08,-4.712058e+12
2,1.0,0.502130,0.555912,0.729567,-19046.0,0.252134,0.279140,0.366337,-9563.564279,0.309038,...,0.171798,0.225464,-5885.942404,0.295894,-7724.580288,2.016572e+08,0.388325,-10137.567875,2.646504e+08,-6.908939e+12
3,1.0,0.502130,0.650442,0.510853,-19005.0,0.252134,0.326606,0.256514,-9542.976957,0.423074,...,0.275185,0.216129,-8040.528832,0.169746,-6314.981929,2.349331e+08,0.133318,-4959.747997,1.845150e+08,-6.864416e+12
4,1.0,0.502130,0.322738,0.510853,-19932.0,0.252134,0.162057,0.256514,-10008.451286,0.104160,...,0.033616,0.053210,-2076.117157,0.084225,-3286.224555,1.282190e+08,0.133318,-5201.667828,2.029540e+08,-7.918677e+12


Just as a sanity check, I'll calculate the correlations between those new engineered features and the target.

In [18]:
poly_feats_train['TARGET'] = train_num['TARGET']

poly_feats_train.corr()['TARGET'].sort_values()

EXT_SOURCE_2 EXT_SOURCE_3                -0.194235
EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3   -0.189593
EXT_SOURCE_2^2 EXT_SOURCE_3              -0.176589
EXT_SOURCE_2 EXT_SOURCE_3^2              -0.171729
EXT_SOURCE_1 EXT_SOURCE_2                -0.166538
EXT_SOURCE_1 EXT_SOURCE_3                -0.164933
EXT_SOURCE_2                             -0.160303
EXT_SOURCE_3                             -0.157397
EXT_SOURCE_1 EXT_SOURCE_2^2              -0.156791
EXT_SOURCE_1 EXT_SOURCE_3^2              -0.151139
EXT_SOURCE_2^2                           -0.149502
EXT_SOURCE_3^2                           -0.142517
EXT_SOURCE_2^3                           -0.140217
EXT_SOURCE_1^2 EXT_SOURCE_2              -0.139696
EXT_SOURCE_1^2 EXT_SOURCE_3              -0.139051
EXT_SOURCE_2 DAYS_BIRTH^2                -0.132844
EXT_SOURCE_3^3                           -0.128569
EXT_SOURCE_3 DAYS_BIRTH^2                -0.127602
EXT_SOURCE_1                             -0.099152
EXT_SOURCE_1 DAYS_BIRTH^2      

Cool! Some of those new features are highly correlated with the target. This can be very useful when we start to apply machine learning. Let's concatenate all the data.

In [19]:
poly_feats_train.drop(['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'], 
                      axis=1, inplace=True)

poly_feats_test.drop(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'], 
                     axis=1, inplace=True)

In [20]:
train_full = pd.concat([train_num, train_cat, poly_feats_train], axis=1)
test_full = pd.concat([test_num, test_cat, poly_feats_test], axis=1)

In [21]:
train_full.to_csv('train_eng.csv', index=False)
test_full.to_csv('test_eng.csv', index=False)

In the next notebook I'll try to use the featuretools lib, which can help me to create a lot of features using automated feature engineering. 

## Pipeline for linear models

For linear models, it is best to create dummies for the categorical variables. For that, I'll use pandas.get_dummies function.

In [22]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PAYMENT_DELAY,OVERPAY,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,1.0,20.421053,0.000000,24.000000,15.000000
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,7.160000,0.000000,10.107143,5.785714
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.666667,0.000000,3.750000,2.250000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,19.375000,0.000000,12.000000,8.650000
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,3.636364,452.384318,15.333333,8.969697


In [23]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PAYMENT_DELAY,OVERPAY,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.285714,0.000000,4.000000,1.444444
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,0.0,0.0,0.0,0.0,3.0,23.555556,0.000000,11.700000,7.200000
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,0.0,0.0,0.0,1.0,4.0,5.180645,1157.662742,21.388889,15.305556
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.000000,622.550708,20.354839,10.129032
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,12.250000,0.000000,12.000000,5.846154


In [24]:
# Storing ID and target information

ID_train = train['SK_ID_CURR']
ID_test = test['SK_ID_CURR']
target = train['TARGET']

In [25]:
train.drop(['SK_ID_CURR', 'TARGET'], axis=1, inplace=True)
test.drop('SK_ID_CURR', axis=1, inplace=True)

In [26]:
ID_train.shape

(307511,)

In [27]:
test.shape

(48744, 124)

In [28]:
train_num = train.select_dtypes(exclude='object')
train_cat = train.select_dtypes(include='object')

test_num = test.select_dtypes(exclude='object')
test_cat = test.select_dtypes(include='object')

train_num = fill_numerical_missings(train_num, 'mean')
test_num = fill_numerical_missings(test_num, 'mean')

train_cat = fill_categorical_missings(train_cat)
test_cat = fill_categorical_missings(test_cat)

train_numcat = pd.concat([train_num, train_cat], axis=1)
test_numcat = pd.concat([test_num, test_cat],axis=1)

In [29]:
train_test = train_numcat.append(test_numcat)
poly_feats_train_test = poly_feats_train.append(poly_feats_test).drop('1', axis=1)

In [30]:
train_test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,reg oper account,block of flats,Panel,No
3,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,reg oper account,block of flats,Panel,No


In [31]:
poly_feats_train_test.head()

,EXT_SOURCE_1^2,EXT_SOURCE_1 EXT_SOURCE_2,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_1 DAYS_BIRTH,EXT_SOURCE_2^2,EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_3^2,EXT_SOURCE_3 DAYS_BIRTH,DAYS_BIRTH^2,...,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3 DAYS_BIRTH^2,DAYS_BIRTH^3
0,0.006895,0.021834,0.011573,-785.612748,0.069142,0.036649,-2487.756636,0.019426,-1318.634256,89510521.0,...,0.018181,0.009637,-654.152107,0.005108,-346.733022,2.353667e+07,0.002707,-183.785678,1.247560e+07,-8.468590e+11
1,0.096887,0.193685,0.159012,-5218.396475,0.387190,0.317876,-10431.950422,0.260971,-8564.448972,281065225.0,...,0.240927,0.197797,-6491.237078,0.162388,-5329.192190,1.748916e+08,0.133318,-4375.173647,1.435830e+08,-4.712058e+12
2,0.252134,0.279140,0.366337,-9563.564279,0.309038,0.405575,-10587.901540,0.532268,-13895.327191,362750116.0,...,0.171798,0.225464,-5885.942404,0.295894,-7724.580288,2.016572e+08,0.388325,-10137.567875,2.646504e+08,-6.908939e+12
3,0.252134,0.326606,0.256514,-9542.976957,0.423074,0.332280,-12361.644326,0.260971,-9708.759482,361190025.0,...,0.275185,0.216129,-8040.528832,0.169746,-6314.981929,2.349331e+08,0.133318,-4959.747997,1.845150e+08,-6.864416e+12
4,0.252134,0.162057,0.256514,-10008.451286,0.104160,0.164872,-6432.819536,0.260971,-10182.320126,397284624.0,...,0.033616,0.053210,-2076.117157,0.084225,-3286.224555,1.282190e+08,0.133318,-5201.667828,2.029540e+08,-7.918677e+12


In [32]:
test_num.shape

(48744, 108)

In [33]:
# merging train_test with poly_feats_train_test
train_test = pd.concat([train_test, poly_feats_train_test], axis=1)

train_test = pd.get_dummies(train_test)
train_test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0,0,0,0,0,0,1,0,1,0
1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0,1,0,0,0,0,0,0,1,0
2,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,0,0,0,0,0,1,0,0,1,0
3,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,0,0,0,0,0,1,0,0,1,0
4,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,0,0,0,0,0,1,0,0,1,0


In [34]:
train_dummied = train_test.iloc[:ID_train.shape[0], :]
test_dummied = train_test.iloc[ID_train.shape[0]:, :]

In [35]:
train_dummied.shape

(307511, 278)

In [36]:
test_dummied.shape

(48744, 278)

In [37]:
ID_train.shape

(307511,)

In [38]:
ID_test.shape

(48744,)

In [39]:
train_final = pd.concat([ID_train, target, train_dummied], axis=1)
test_final = pd.concat([ID_test, test_dummied], axis=1)

In [40]:
train_final.to_csv('train_eng_linear.csv', index=False)
test_final.to_csv('test_eng_linear.csv', index=False)